### Package Installs and Imports

In [ ]:
%conda install -c conda-forge tweepy

In [1]:
import tweepy
import json

### Twitter Authetications and Scraping

In [2]:
with open('credentials.json', 'r') as ifile:
    creds = json.load(ifile)

In [13]:
api_key = creds['api_key']
api_secret = creds['api_key_secret']
bearer_token = creds['bearer_token']
access_token = creds['access_token']
access_token_secret = creds['access_token_secret']

auth = tweepy.OAuth1UserHandler(api_key, api_secret, access_token, access_token_secret)
api = tweepy.API(auth)

In [14]:
user_id = 'realDonaldTrump'
tweets = api.user_timeline(screen_name='realDonaldTrump', count=10, include_rts=False, tweet_mode='extended')
oldest_id = tweets[-1].id
all_tweets = []
all_tweets.extend(tweets)
while True:
    tweets = api.user_timeline(screen_name='realDonaldTrump', count=200, include_rts=False, max_id=oldest_id - 1, tweet_mode='extended')
    if len(tweets) == 0:
        break
    oldest_id = tweets[-1].id
    all_tweets.extend(tweets)
    print(f"Downloaded {len(all_tweets)} tweets so far")

Downloaded 147 tweets so far
Downloaded 277 tweets so far
Downloaded 408 tweets so far
Downloaded 553 tweets so far
Downloaded 667 tweets so far
Downloaded 792 tweets so far
Downloaded 925 tweets so far
Downloaded 1091 tweets so far
Downloaded 1268 tweets so far
Downloaded 1382 tweets so far
Downloaded 1517 tweets so far
Downloaded 1628 tweets so far
Downloaded 1739 tweets so far
Downloaded 1858 tweets so far
Downloaded 1966 tweets so far
Downloaded 2078 tweets so far
Downloaded 2096 tweets so far


In [15]:
for info in all_tweets[:3]:
    print(info.full_text)

To all of those who have asked, I will not be going to the Inauguration on January 20th.
The 75,000,000 great American Patriots who voted for me, AMERICA FIRST, and MAKE AMERICA GREAT AGAIN, will have a GIANT VOICE long into the future. They will not be disrespected or treated unfairly in any way, shape or form!!!
https://t.co/csX07ZVWGe
